<a href="https://colab.research.google.com/github/kierandesmond/text-classification/blob/master/EmbankAI_PII_Spanish_PII_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: I want to train an nlp model to detect PII from Spanish prompts, I have a csv file example that I can upload

!pip install transformers datasets evaluate seqeval
!pip install optuna
from seqeval import scheme
from seqeval.scheme import IOB2
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import shutil
import datetime



import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
import shutil

# Load your CSV data
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/Test-Data-Embank/realistic_synthetic_pii_dataset.csv')

hf_dataset = Dataset.from_pandas(data.rename(columns={"Text": "text", "Label": "labels"}))

print(data.head())



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=0d135cc89bbc292b906b3702dc756dbb93e6bcbef4c58d15cae879e99edccf9c
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10

In [ ]:

# Ensure proper column renaming
data = data.rename(columns={"Text": "text", "Label": "labels"})
hf_dataset = Dataset.from_pandas(data)

# Split into train/test/validation
train_testvalid = hf_dataset.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)
split_datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "validation": test_valid["train"]
})

from transformers import PreTrainedTokenizerFast

def preprocess_and_align_labels(examples, tokenizer):
    aligned_labels = []
    for text, labels in zip(examples["text"], examples["labels"]):
        # Split text into words (if not already split)
        pretokenized_text = list(text)  # Ensure text is split into characters/words
        label_list = labels.split()  # Ensure labels are split into words

        # Tokenize with `is_split_into_words=True`
        tokens = tokenizer(pretokenized_text, truncation=True, is_split_into_words=True)
        word_ids = tokens.word_ids()  # Map tokens to words

        # Align labels
        aligned_label = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:  # Special tokens
                aligned_label.append(-100)
            # Check if word_idx is within the bounds of label_list
            elif word_idx != previous_word_idx and word_idx < len(label_list):
                aligned_label.append(label2id.get(label_list[word_idx], -100))  # Map labels to IDs
            else:
                aligned_label.append(-100)  # Inside a word, skip additional tokens
            previous_word_idx = word_idx

        aligned_labels.append(aligned_label)

    return aligned_labels





def preprocess_data(examples):
    # Pre-tokenize the text by splitting it into words
    pretokenized_texts = [list(text) for text in examples["text"]]  # Convert each string to a list of characters/words

    # Tokenize while considering the pre-tokenized input
    tokenized_inputs = tokenizer(
        pretokenized_texts,
        truncation=True,
        padding=True,
        is_split_into_words=True
    )

    # Align labels with tokens
    aligned_labels = preprocess_and_align_labels(examples, tokenizer)
    tokenized_inputs["labels"] = aligned_labels

    return tokenized_inputs




# Convert data into Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data)

# Split dataset into train/test/validation
split_datasets = hf_dataset.train_test_split(test_size=0.2, seed=42)
print("Sample Texts:", split_datasets["train"]["text"][:5])
print("Sample Labels:", split_datasets["train"]["labels"][:5])

test_valid = split_datasets["test"].train_test_split(test_size=0.5, seed=42)
# Ensure all text inputs are strings
print("Gets here..")
split_datasets = split_datasets.map(
    lambda examples: {
        "text": [str(x) if not isinstance(x, str) else x for x in examples["text"]],
        "labels": examples["labels"]
    },
    batched=True
)
print("Dataset split and prepped?")
# Tokenize and align
tokenized_dataset = split_datasets.map(preprocess_data, batched=True)




Sample Texts: ['X9573502G', '2972-29-825881623', '3283-73-584264233', 'L60923828F', 'E76537608']
Sample Labels: ['B-NIE', 'B-CCC', 'B-CCC', 'B-DNI_L', 'B-VAT']
Gets here..


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset split and prepped?


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:


# Load a pre-trained model and tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import DatasetDict

# Initialize tokenizer and model
model_checkpoint = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

id2label = {  # Your label mapping
    0: "B-NIF", 1: "I-NIF", 2: "B-NIE", 3: "I-NIE", 4: "B-CIF", 5: "I-CIF",
    6: "B-Passport", 7: "I-Passport", 8: "B-DNI", 9: "I-DNI", 10: "B-DNI_K", 11: "I-DNI_K",
    12: "B-DNI_L", 13: "I-DNI_L", 14: "B-VAT", 15: "I-VAT", 16: "B-SocialSecurity",
    17: "I-SocialSecurity", 18: "B-CCC", 19: "I-CCC", 20: "O"
}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

special_tokens = ['NIF', 'NIE', 'CIF', 'Passport', 'DNI', 'DNI_K', 'DNI_L', 'VAT', 'SocialSecurity', 'CCC']
num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
model.resize_token_embeddings(len(tokenizer))


# Instead, apply the tokenization and splitting as follows:
tokenized_datasets = hf_dataset.map(preprocess_data, batched=True)

# Split the tokenized dataset
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)

# Combine splits into a DatasetDict
tokenized_datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "validation": test_valid["train"]
})


from datasets import DatasetDict

# Split dataset
# Access the 'train' dataset within tokenized_dataset for splitting
train_testvalid = tokenized_dataset["train"].train_test_split(test_size=0.2)  # Split 'train' into train/test

test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)  # Split 20% into 10% test and 10% validation

# Combine splits into DatasetDict
tokenized_dataset = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "validation": test_valid["train"]
})

# Metrics
metric = evaluate.load("seqeval")
custom_scheme = [
    "B-NIF", "I-NIF", "B-NIE", "I-NIE", "B-CIF", "I-CIF",
    "B-Passport", "I-Passport", "B-DNI", "I-DNI", "B-DNI_K", "I-DNI_K",
    "B-DNI_L", "I-DNI_L", "B-VAT", "I-VAT", "B-SocialSecurity",
    "I-SocialSecurity", "B-CCC", "I-CCC", "O"
]
custom_scheme_string = ','.join(custom_scheme)  # Join elements with commas

unmatched_labels = set(data['labels']) - set(label2id.keys())
print("Unmatched labels:", unmatched_labels)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists for per-label metrics calculation
    flat_true_predictions = [label for sublist in true_predictions for label in sublist]
    flat_true_labels = [label for sublist in true_labels for label in sublist]

    # Compute metrics
    precision = precision_score(flat_true_labels, flat_true_predictions, average="weighted", zero_division=0)
    recall = recall_score(flat_true_labels, flat_true_predictions, average="weighted", zero_division=0)
    f1 = f1_score(flat_true_labels, flat_true_predictions, average="weighted", zero_division=0)
    accuracy = accuracy_score(flat_true_labels, flat_true_predictions)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
    }


# Define the model_init function
def model_init(trial):
    return AutoModelForTokenClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id
    )

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # Adjust as needed
    weight_decay=0.01,
)




Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Unmatched labels: set()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:

# Create a Trainer instance
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
# Define the hyperparameter space with trial suggestions
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 2e-5, 3e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 4]),
    }

# Reinitialize the Trainer without the 'model' argument
trainer = Trainer(
    args=training_args,  # Remove 'model'
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,  # Ensure data_collator is configured with tokenizer
    compute_metrics=compute_metrics,
    # tokenizer=tokenizer,  # Remove this line
    model_init=model_init  # Keep model_init
)

#TODO add hyperparameters for better accuracy
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 2e-5, 3e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2, 3, 4]),
    }

# Perform hyperparameter search
best_run = trainer.hyperparameter_search(
    hp_space=lambda trial: hp_space(trial),  # Pass a lambda function
    direction="maximize",
    backend="optuna",
    n_trials=10,
)

new_training_args = TrainingArguments(
    output_dir="./results",
    **best_run.hyperparameters  # Pass the best hyperparameters
)

# Update training arguments with best hyperparameters
#training_args.update(best_run.hyperparameters)
# Use the new training_args object for the final training
trainer = Trainer(
    model=model,  # Or model_init if you have it
    args=new_training_args,
     train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,  # Ensure data_collator is configured with tokenizer
    compute_metrics=compute_metrics
    # ... (rest of your trainer configuration) ...
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model("./pii_detection_model")


# Generate a datetime stamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Create a versioned output directory name
output_dir = f"./pii_detection_model_{timestamp}"

# Save the trained model to the versioned directory
trainer.save_model(output_dir)

tokenizer.save_pretrained(f"/content/drive/My Drive/Test-Data-Embank/pii_detection_model_{timestamp}")
model.save_pretrained(f"/content/drive/My Drive/Test-Data-Embank/pii_detection_model_{timestamp}")

# Copy the versioned directory to your Google Drive
shutil.copytree(output_dir, f"/content/drive/My Drive/Test-Data-Embank/pii_detection_model_{timestamp}2")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-01-18 11:37:49,559] A new study created in memory with name: no-name-ac9f9ec9-9ad9-4d40-b2b5-ce6a644dbd0f
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


wandb: Currently logged in as: kieran-desmond (kieran-desmond-embankai). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.520426,0.770475,0.762500,0.731763,0.762500
2,No log,0.376425,0.786100,0.787500,0.743250,0.787500
3,No log,0.379346,0.781764,0.775000,0.774302,0.775000


[I 2025-01-18 12:03:18,051] Trial 0 finished with value: 3.106066892489195 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 3}. Best is trial 0 with value: 3.106066892489195.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁█▄
eval/f1,▁▃█
eval/loss,█▁▁
eval/precision,▁█▆
eval/recall,▁█▄
eval/runtime,▂▁█
eval/samples_per_second,▇█▁
eval/steps_per_second,▇█▁
train/epoch,▁▅██
train/global_step,▁▅██
eval/accuracy,0.775


...

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.775


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.432507,0.800234,0.787500,0.766836,0.787500
2,No log,0.396886,0.801312,0.795833,0.758388,0.795833


[I 2025-01-18 15:13:51,531] Trial 9 finished with value: 3.151367045412164 and parameters: {'learning_rate': 1e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 2}. Best is trial 2 with value: 3.240688140341181.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss


FileExistsError: [Errno 17] File exists: '/content/drive/My Drive/Test-Data-Embank/pii_detection_model_20250118_155020'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')